In [ ]:
import argparse
import glob
import os
import json
import time
import logging
import random
import re
from itertools import chain
from string import punctuation

import nltk
nltk.download('punkt')
from nltk.tokenize import sent_tokenize
import csv

import pandas as pd
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
import pytorch_lightning as pl

# eval packages
import textwrap
from tqdm.auto import tqdm
from sklearn import metrics


from transformers import (
    AdamW,
    T5ForConditionalGeneration,
    T5Tokenizer,
    get_linear_schedule_with_warmup
)

def set_seed(seed):
  random.seed(seed)
  np.random.seed(seed)
  torch.manual_seed(seed)
  if torch.cuda.is_available():
    torch.cuda.manual_seed_all(seed)

set_seed(42)

def get_folds(csv_path):
    #TODO: check for URLs and possibly turn them into text!?
    val_inputs = []
    val_targets = []
    train_inputs = []
    train_targets = []

    REPLACE_NO_SPACE = re.compile("[.;:!\'?,\"()\[\]]")
    REPLACE_WITH_SPACE = re.compile("(<br\s*/><br\s*/>)|(\-)|(\/)")

    LABELS = ["unproblematisch", "problematisch"]

    with open(csv_path, encoding="utf-8") as f_source:
        reader = csv.DictReader(f_source)
        for row in reader:
            line = "classification: " + row["text"]
            line = REPLACE_NO_SPACE.sub("", line)
            line = REPLACE_WITH_SPACE.sub("", line)
            line = line

            target = LABELS[int(float(row['label']))]

            if int(row["ten_folds"]) <= 7:
                train_inputs.append(line)
                train_targets.append(target)
            else:
                val_inputs.append(line)
                val_targets.append(target)
        
        return train_inputs, train_targets, val_inputs, val_targets

class T5FineTuner(pl.LightningModule):
  def __init__(self, hparams):
    super(T5FineTuner, self).__init__()
    # self.hparams.save_hyperparameters(hparams)
    self.save_hyperparameters(hparams)
    
    self.model = T5ForConditionalGeneration.from_pretrained(hparams.model_name_or_path)
    self.tokenizer = T5Tokenizer.from_pretrained(hparams.tokenizer_name_or_path)
    # self.train_dataset = train_dataset
    # self.val_dataset = val_dataset
    
  def is_logger(self):
    return self.trainer.global_rank <= 0
  

  def forward(
      self, input_ids, attention_mask=None, decoder_input_ids=None, decoder_attention_mask=None, labels=None
  ):
    return self.model(
        input_ids,
        attention_mask=attention_mask,
        decoder_input_ids=decoder_input_ids,
        decoder_attention_mask=decoder_attention_mask,
        labels=labels,
        # lm_labels=lm_labels,
    )

  def _step(self, batch):
    labels = batch["target_ids"]
    labels[labels[:, :] == self.tokenizer.pad_token_id] = -100

    outputs = self(
        input_ids=batch["source_ids"],
        attention_mask=batch["source_mask"],
        labels=labels,
        # lm_labels=lm_labels,
        decoder_attention_mask=batch['target_mask']
    )

    loss = outputs[0]

    return loss
  
  def get_accuracy(self, batch):
    labels = batch["target_ids"]

    outs = self.model.generate(input_ids=batch["source_ids"], 
                                attention_mask=batch["source_mask"], 
                                max_length=2)

    dec = [self.tokenizer.decode(ids) for ids in outs]
    target = [self.tokenizer.decode(label) for label in labels]

    new_outputs = [s[6:] for s in dec]
    new_targets = [s[:-4] for s in target]

    accuracy_score = metrics.accuracy_score(new_targets, new_outputs)
    f1 = metrics.f1_score(new_targets, new_outputs, average="micro")
    # rec = metrics.recall_score(new_targets, new_outputs, average="micros")
    return accuracy_score, f1
  # def computer_accuracy(self):
  #   self.model.model.eval()

  def training_step(self, batch, batch_idx):
    loss = self._step(batch)
    self.log("train_loss", loss)
    return {"loss": loss}
  
  def training_epoch_end(self, outputs):
    avg_train_loss = torch.stack([x["loss"] for x in outputs]).mean()
    tensorboard_logs = {"avg_train_loss": avg_train_loss}
    self.log("avg_train_loss", avg_train_loss)
    
  def validation_step(self, batch, batch_idx):
    loss = self._step(batch)
    accuracy, f1 = torch.tensor(self.get_accuracy(batch))
    self.log("val_loss", loss, logger=True)
    self.log("val_accuracy", accuracy, logger=True)
    return {"val_loss": loss, "val_accuracy": accuracy}
  
  def validation_epoch_end(self, outputs):
    avg_loss = torch.stack([x["val_loss"] for x in outputs]).mean()
    avg_accuracy = torch.stack([x["val_accuracy"] for x in outputs]).mean()
    # tensorboard_logs = {"val_loss": avg_loss}
    self.log("avg_val_loss", avg_loss)
    self.log("avg_val_accuracy", avg_accuracy)
    # self.log("log", tensorboard_logs)
    # self.log("progress_bar", tensorboard_logs)
    # self.log({"avg_val_loss": avg_loss, 
    #           "log": tensorboard_logs,
    #           'progress_bar': tensorboard_logs}, logger=True, prog_bar=True)
    return {"avg_val_loss": avg_loss, "avg_val_accuracy": avg_accuracy}

  def configure_optimizers(self):
    "Prepare optimizer and schedule (linear warmup and decay)"

    model = self.model
    no_decay = ["bias", "LayerNorm.weight"]
    optimizer_grouped_parameters = [
        {
            "params": [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)],
            "weight_decay": self.hparams.weight_decay,
        },
        {
            "params": [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)],
            "weight_decay": 0.0,
        },
    ]
    optimizer = AdamW(optimizer_grouped_parameters, lr=self.hparams.learning_rate, eps=self.hparams.adam_epsilon)
    self.opt = optimizer
    return [optimizer]
  
  def optimizer_step(self, 
                      epoch,
                      batch_idx,
                      optimizer,
                      optimizer_idx,
                      second_order_closure=None,
                      on_tpu=None,
                      using_native_amp=None,
                      using_lbfgs=None):
    # if self.trainer.use_tpu:
    #   xm.optimizer_step(optimizer)
    # else:
    optimizer.step(closure=second_order_closure)
    optimizer.zero_grad()
    self.lr_scheduler.step()
  
  # def closure(self):
  #   return "closure"
  
  def get_tqdm_dict(self):
    tqdm_dict = {"loss": "{:.3f}".format(self.trainer.avg_loss), "lr": self.lr_scheduler.get_last_lr()[-1]}

    return tqdm_dict

  def train_dataloader(self):
    # train_dataset = get_dataset(tokenizer=self.tokenizer, type_path="train", args=self.hparams)
    dataloader = DataLoader(self.hparams.train_dataset, batch_size=self.hparams.train_batch_size, drop_last=True, shuffle=True, num_workers=4)
    t_total = (
        (len(dataloader.dataset) // (self.hparams.train_batch_size * max(1, self.hparams.n_gpu)))
        // self.hparams.gradient_accumulation_steps
        * float(self.hparams.num_train_epochs)
    )
    scheduler = get_linear_schedule_with_warmup(
        self.opt, num_warmup_steps=self.hparams.warmup_steps, num_training_steps=t_total
    )
    self.lr_scheduler = scheduler
    return dataloader

  def val_dataloader(self):
    # val_dataset = get_dataset(tokenizer=self.tokenizer, type_path="val", args=self.hparams)
    return DataLoader(self.hparams.val_dataset, batch_size=self.hparams.eval_batch_size, num_workers=4)

class RPDataset(Dataset):
    def __init__(self, tokenizer, inputs, outputs, max_len=512):
        self.max_len = max_len
        self.tokenizer = tokenizer

        self.inputs = inputs
        self.outputs = outputs

        self.tokenized_inputs = tokenizer.batch_encode_plus(
            inputs, max_length=max_len, padding=True, truncation=True, return_tensors="pt"
        )
        self.tokenized_targets = tokenizer.batch_encode_plus(
            outputs, max_length=2, padding=True, truncation=True, return_tensors="pt"
        )
    
    def __len__(self):
        return len(self.inputs)
    
    def __getitem__(self, index):
        # tokenize input and output
        
        source_ids = self.tokenized_inputs["input_ids"][index].squeeze()
        target_ids = self.tokenized_targets["input_ids"][index].squeeze()

        src_mask    = self.tokenized_inputs["attention_mask"][index].squeeze()  # might need to squeeze
        target_mask = self.tokenized_targets["attention_mask"][index].squeeze()  # might need to squeeze

        return {"source_ids": source_ids, "source_mask": src_mask, "target_ids": target_ids, "target_mask": target_mask}

def get_folds(csv_path):
    #TODO: check for URLs and possibly turn them into text!?
    val_inputs = []
    val_targets = []
    train_inputs = []
    train_targets = []

    REPLACE_NO_SPACE = re.compile("[.;:!\'?,\"()\[\]]")
    REPLACE_WITH_SPACE = re.compile("(<br\s*/><br\s*/>)|(\-)|(\/)")

    LABELS = ["unproblematisch", "problematisch"]

    with open(csv_path, encoding="utf-8") as f_source:
        reader = csv.DictReader(f_source)
        for row in reader:
            line = "classification: " + row["text"]
            line = REPLACE_NO_SPACE.sub("", line)
            line = REPLACE_WITH_SPACE.sub("", line)
            line = line

            target = LABELS[int(float(row['label']))]

            if int(row["ten_folds"]) <= 7:
                train_inputs.append(line)
                train_targets.append(target)
            else:
                val_inputs.append(line)
                val_targets.append(target)
        
        return train_inputs, train_targets, val_inputs, val_targets
        

